In [1]:
import requests
from bs4 import BeautifulSoup
import sqlite3

In [2]:
#Paso 2: Obtener el contenido de la página web

url = "https://www.avalanches.org/fatalities/"
response = requests.get(url)
html = response.text


In [3]:
#Paso 3: Analizar el contenido HTML con Beautiful Soup

soup = BeautifulSoup(html, "html.parser")


In [4]:
#Paso 4: Extraer la información relevante

data_rows = soup.select("table tr")[1:]  # Excluye la primera fila de encabezados

records = []
for row in data_rows:
    columns = row.find_all("td")
    location = columns[0].text.strip()
    country = columns[1].text.strip()
    date = columns[2].text.strip()
    problem_1 = columns[3].text.strip()
    problem_2 = columns[4].text.strip()
    dead = columns[5].text.strip()
    group_size = columns[6].text.strip()
    atype = columns[7].text.strip()

    records.append((location, country, date, problem_1, problem_2, dead, group_size, atype))



In [5]:
#Paso 5: Conectar y escribir en la base de datos SQLite

conn = sqlite3.connect("avalanche_fatalities_22_23.db")
cursor = conn.cursor()

# Crea la tabla si no existe
cursor.execute('''
    CREATE TABLE IF NOT EXISTS fatalities (
        id INTEGER PRIMARY KEY,
        location TEXT,
        country TEXT,
        date TEXT,
        avalanche_problem_1 TEXT,
        avalanche_problem_2 TEXT,
        dead TEXT,
        group_size TEXT,
        type TEXT
    )
''')

# Inserta los registros en la tabla
cursor.executemany('''
    INSERT INTO fatalities (location, country, date, avalanche_problem_1, avalanche_problem_2, dead, group_size, type)
    VALUES (?, ?, ?, ?, ?, ?, ?, ?)
''', records)

conn.commit()
conn.close()

In [6]:
import csv     
 
# Nombre del archivo CSV
csv_filename = "avalanche_fatalities_22_23.csv"

# Encabezados para el archivo CSV
csv_headers = ["Location", "Country", "Date", "Avalanche Problem 1", "Avalanche Problem 2", "Dead", "Group Size", "Type"]

# Escribir los registros en el archivo CSV
with open(csv_filename, "w", newline="", encoding="utf-8") as csv_file:
    csv_writer = csv.writer(csv_file)
    
    # Escribir los encabezados
    csv_writer.writerow(csv_headers)
    
    # Escribir los registros
    for record in records:
        csv_writer.writerow(record)

